In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Membaca dataset
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/final_data.csv')
df.sample(2)

,Unnamed: 0,sentence_id,new_sentence,type,sentence_length,cleaned_sentence,corrected_sentence,token_words,stop_words,stemmed_words,lemmatized_words,word_count,sentiment_score,label_type
28338,28849,INDEDU23952,"BS/BA degree , equivalent experience.",Education,38,"BS/BA degree , equivalent experience.","BS/BA degree , equivalent experience","['degree', 'equivalent', 'experience']","['degree', 'equivalent', 'experience']","['degre', 'equival', 'experi']","['degree', 'equivalent', 'experience']",5,0.00,0
10241,10413,HONREQ15829,Preferred to have candidate having prior worki...,Requirement,101,Preferred to have candidate having prior worki...,Preferred to have candidate having prior worki...,"['Preferred', 'to', 'have', 'candidate', 'havi...","['Preferred', 'candidate', 'prior', 'working',...","['prefer', 'candid', 'prior', 'work', 'experi'...","['Preferred', 'candidate', 'prior', 'working',...",14,0.05,2


1. SET UP DEVELOPMENT ENVIRONTMENT

In [ ]:
# !pip install pytesseract transformers datasets evaluate rouge-score nltk tensorboard py7zr --upgrade
!pip install pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!git config --global user.email "fauzan.rizki.akmarina2@gmail.com"
!git config --global user.name "fauzanadnans"


In [ ]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

2. PREPARE DATASET

In [ ]:
#Split dataset
df2 = df.copy()
from datasets import Dataset, DatasetDict

# Assuming you have loaded your pandas DataFrame into a variable 'df'
data = pd.DataFrame({
    'text': df2['corrected_sentence'],
    'label': df2['label_type']
})

# Split the data into train, test, and validation sets
train_data, eval_test_data = train_test_split(data, test_size=0.3, random_state=42)
test_data, validation_data = train_test_split(eval_test_data, test_size=0.5, random_state=42)

# Convert each split into a Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
validation_dataset = Dataset.from_pandas(validation_data)

# Kurangi dataset train dari 41,300 ke 5,000
train_dataset = train_dataset.shuffle(seed=42).select(range(3000))
test_dataset = test_dataset.shuffle(seed=42).select(range(1000))
validation_dataset = validation_dataset.shuffle(seed=42).select(range(1000))

# Buat dataset dictionary
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

# Cek jumlah dataset setelah dikurangi
print(dataset)



In [ ]:
# Convert HF Dataset ke Pandas DataFrame sebelum menyimpan
train_dataset.to_pandas().to_csv("train.csv", index=False)
test_dataset.to_pandas().to_csv("test.csv", index=False)
validation_dataset.to_pandas().to_csv("validation.csv", index=False)

print("Semua dataset berhasil disimpan sebagai CSV!")


In [ ]:
from google.colab import files

# Download file CSV ke lokal
files.download("train.csv")
files.download("test.csv")
files.download("validation.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
print(os.getcwd())  # Menampilkan direktori saat ini


In [ ]:
dataset['train'][1]

{'text': 'i years of Experience in Automation and majorly in Selenium and Java.',
 'label': 1,
 '__index_level_0__': 51099}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
#convert label from int to string
import pandas as pd
from datasets import Dataset
import random

dataset['train'] = dataset['train'].shuffle(seed=42)

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
dataset.clear()
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

3. Fine-tune and evaluate FLAN-T5


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-small"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("accuracy")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-imdb-text-classification"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=82,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=2,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=1000,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    #push_to_hub=False,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
trainer.train()


In [ ]:
# Direktori lokal
local_path = "./flan-t5-finetuned-nautix"

# Direktori di Google Drive
gdrive_path = "/content/drive/My Drive/flan-t5-finetuned"

# Simpan model dan tokenizer di dua lokasi
trainer.save_model(local_path)  # Menyimpan di direktori lokal
tokenizer.save_pretrained(local_path)

trainer.save_model(gdrive_path)  # Menyimpan di Google Drive
tokenizer.save_pretrained(gdrive_path)


In [ ]:
model.push_to_hub("fauzanadnans/flan-t5-finetuned-classification")
tokenizer.push_to_hub("fauzanadnans/flan-t5-finetuned-classification")


# METRIKS EVALUASI

In [ ]:

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


eval_results = trainer.evaluate(tokenized_dataset["test"])
eval_results2 = trainer.evaluate(tokenized_dataset["train"])
eval_results3 = trainer.evaluate(tokenized_dataset["validation"])


print(f"Test: {eval_results}")
print(f":Train {eval_results}")
print(f"Validation: {eval_results}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


NameError: name 'trainer' is not defined

In [ ]:
from sklearn.metrics import classification_report
import torch

def evaluate_and_report(model, tokenizer, dataset):
    # Ambil input_ids sebagai masukan model
    inputs = {
        "input_ids": torch.tensor(dataset["input_ids"]),
        "attention_mask": torch.tensor(dataset["attention_mask"])
    }

    # Generate prediksi dari model
    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Decode hasil prediksi ke teks
    predicted_labels = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Ambil label asli dari dataset
    true_labels = dataset["labels"]

    # Pastikan label dalam bentuk string untuk classification_report
    predicted_labels = [str(label) for label in predicted_labels]
    true_labels = [str(label) for label in true_labels]

    # Print Classification Report
    report = classification_report(true_labels, predicted_labels)
    print(report)

    return report

# Panggil fungsi untuk evaluasi dataset
test_result1 = evaluate_and_report(model, tokenizer, tokenized_dataset["test"])
test_result2 = evaluate_and_report(model, tokenizer, tokenized_dataset["test"])

print(f"Test Metrics: \n{test_result1}")


In [ ]:
!pip install transformers datasets torch tqdm scikit-learn


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from sklearn.metrics import classification_report
from tqdm.auto import tqdm


In [ ]:
model_path = "./flan-t5-finetuned-nautix" # Sesuaikan dengan path modelmu
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
